# part c-1) Neural network with tensorflow (basic)
We are going to make neural network with 3 inputs and 2 outputs. Relu function is used for non linearity at each layer. 2 hidden layers are used -one consisting of 5 neurons and second hidden layer consisting of four neurons.We are tryint to predict more than one value.
we use dtwo non linear functions-
1. 5x1^2+x2^2+x3^2+5x1+x2+x3
2. 4x1^2+x2^2+1x3^2+4x1+x2+1x3

In [ ]:
import numpy as np
import random
from numpy import array
from numpy.random import uniform
from numpy import hstack
import torch
import torch.nn as nn
import tensorflow as tf

In [ ]:
#creating a class to define linear layers , here input dim represents number of input attributes, num_hidden is number of output neurons  
class Linear:
  def __init__(self,input_dim:int,num_hidden:int):
    self.weights=tf.Variable(np.random.rand(input_dim, num_hidden), dtype=tf.float32)
    self.bias=tf.zeros(num_hidden)

  def __call__(self, x):
    self.x = x
    output = tf.matmul(x, self.weights) + self.bias
    return output

  def backward(self, gradient): # calculating loss function gradient
    self.weights_gradient = tf.matmul(tf.transpose(self.x),gradient)
    self.bias_gradient = tf.reduce_sum(gradient)
    self.x_gradient =  tf.matmul(gradient,tf.transpose(self.weights))
    return self.x_gradient

  def update(self, lr): # updating weights and bias
    self.weights = self.weights - lr * self.weights_gradient
    self.bias = self.bias - lr * self.bias_gradient


In [ ]:
#defining relu function and its derative
class Relu:
  def __call__(self,input_):
    self.input_=input_
    self.output=tf.clip_by_value(self.input_,clip_value_min=0,clip_value_max=self.input_)
    #print(input_,self.output)#torch.clamp(self.input_, min=0)#
    return self.output
  def backward(self,output_gradient):
    self.input_gradient=tf.cast((self.input_ > 0), tf.float32)*output_gradient
    return self.input_gradient



In [ ]:
#creating Non linear model consisting of 3 layers-2 hidden and one output layer consisting of two output neurons.
class MyModel():
   def __init__(self,input_dim:int,num_hidden1:int,num_hidden2:int):
    
    self.linear1=Linear(input_dim, num_hidden1)
    self.relu1=Relu()
    self.linear2=Linear(num_hidden1,num_hidden2)
    self.relu2=Relu()
    self.linear3=Linear(num_hidden2,2)
  
   def __call__(self,x): # forward propagation step
    l1=self.linear1(x)
    r1=self.relu1(l1)
    l2=self.linear2(r1)
    r2=self.relu2(l2)
    l3=self.linear3(r2)
    return l3

   def backward(self,output_gradient): # Backward Propagation step
    linear3_gradient=self.linear3.backward(output_gradient)
    relu_gradient1=self.relu2.backward(linear3_gradient)
    linear2_gradient=self.linear2.backward(relu_gradient1)
    relu_gradient2=self.relu1.backward(linear2_gradient)
    linear1_gradient=self.linear1.backward(relu_gradient2)
    return linear1_gradient
  
   def update(self,lr): # updating weights for each layer
    self.linear3.update(lr)
    self.linear2.update(lr)
    self.linear1.update(lr)




In [ ]:
#defining loss function-mse and its derivative  
class MSE:
  def __call__(self,y_pred,y_true):
    self.y_pred=y_pred
    self.y_true=y_true
    return tf.reduce_mean(tf.square(y_true-y_pred))
  def backward(self):
   n=self.y_true.shape[0]
   self.gradient=2.*(self.y_pred-self.y_true)/n
   return self.gradient

In [ ]:
n=400  # number of input samples
d=3    # number of dimensions
num_hidden1=5 # number of neurons for first hidden layer
num_hidden2=4 # numbe rof neurons for second hidden layer
x= tf.convert_to_tensor(tf.Variable(np.random.rand(400, 3), dtype=tf.float32))
w_true=tf.constant([[5,1,1],[4,1,1]], dtype=tf.float32)
w_true=tf.transpose(w_true)
bias_true=tf.constant([1], dtype=tf.float32)
y=tf.matmul(x**2,w_true)+tf.matmul(x,w_true)+bias_true
model = MyModel(d,num_hidden1,num_hidden2)
loss=MSE()

from typing import Callable
def tensor_fit(x,y,model:Callable,loss:Callable,lr:float,num_epochs: int):
  # optimizer=torch.optim.SGD(model.parameters(),lr=lr)

  for epoch in range(num_epochs):
    y_pred=model(x)
    loss_value=loss(y_pred,y)
    print(f"Epoch{epoch}, loss{loss_value}")
    gradient_from_loss=loss.backward()
    model.backward(gradient_from_loss)
    model.update(lr)

tensor_fit(x,y, model=model, loss=loss, lr=0.005, num_epochs=50) 

Epoch0, loss7.4694390296936035
Epoch1, loss3.2427306175231934
Epoch2, loss2.769052743911743
Epoch3, loss2.411944627761841
Epoch4, loss2.2498865127563477
Epoch5, loss2.123455286026001
Epoch6, loss2.0260655879974365
Epoch7, loss1.9381183385849
Epoch8, loss1.8569602966308594
Epoch9, loss1.779383897781372
Epoch10, loss1.704781174659729
Epoch11, loss1.632514238357544
Epoch12, loss1.5624867677688599
Epoch13, loss1.494553565979004
Epoch14, loss1.4287272691726685
Epoch15, loss1.3649721145629883
Epoch16, loss1.3033154010772705
Epoch17, loss1.2437466382980347
Epoch18, loss1.1862868070602417
Epoch19, loss1.1309254169464111
Epoch20, loss1.0776728391647339
Epoch21, loss1.0265125036239624
Epoch22, loss0.9774410128593445
Epoch23, loss0.930433452129364
Epoch24, loss0.8854737281799316
Epoch25, loss0.8425279259681702
Epoch26, loss0.8015680909156799
Epoch27, loss0.7625506520271301
Epoch28, loss0.7254383563995361
Epoch29, loss0.6901795864105225
Epoch30, loss0.6567544341087341
Epoch31, loss0.62517374753952

In [ ]:
def plot_intereactive_3d(x, y, y_pred=None):
  import plotly.graph_objects as go

  fig = go.Figure()
  fig.add_trace(go.Scatter3d(x = x[:,0],
                    y = x[:,1],
                    z = y.reshape([-1]),
                    opacity=0.5, mode='markers', name='Underlying Function'
                    ))
 
  if y_pred is not None:
    fig.add_trace(go.Scatter3d(x = x[:,0],
                   y = x[:,1],
                   z = y_pred.reshape([-1]),
                   opacity=0.5, mode='markers', name='Predicted Function'
                  ))
    
  fig.update_layout(scene = dict(
                    xaxis_title='X1',
                    yaxis_title='X2',
                    zaxis_title='Y'),
                    width=700,
                    margin=dict(r=20, b=10, l=10, t=10))
  fig.show()

In [ ]:
from sklearn.manifold import TSNE
X_red = TSNE(n_components=2).fit_transform(x)
y_true_red = TSNE(n_components=1).fit_transform(y)
y_pred_red = TSNE(n_components=1).fit_transform(model(x))
print(f'X_red: {X_red.shape}, y_true_red: {y_true_red.shape}, y_pred_red: {y_pred_red.shape}')

plot_intereactive_3d(X_red,y_true_red,y_pred_red)

X_red: (400, 2), y_true_red: (400, 1), y_pred_red: (400, 1)
